In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
def plot_history(hst):
    plt.figure(figsize=(13, 4))

    plt.subplot(1, 2, 1)
    plt.plot(hst.history['loss'], label='train')
    plt.plot(hst.history['val_loss'], label='test')
    plt.title('Loss')

    plt.subplot(1, 2, 2)
    plt.plot([round(100*e, 2) for e in hst.history['sparse_categorical_accuracy']], label='train')
    plt.plot([round(100*e, 2) for e in hst.history['val_sparse_categorical_accuracy']], label='test')
    plt.title('Accuracy')    

    plt.ylim(0, 100)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

N = 30000

x_train = x_train[:N] / 255
x_test = x_test[:N] / 255

y_train = y_train[:N]
y_test = y_test[:N]


x_train.shape, y_train.shape, x_test.shape, y_test.shape

## Base Line

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

#model.summary()

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
plot_history(history)

## Optimizers

In [ ]:
collect = {}

optimizers_list = [
    tf.keras.optimizers.Adam,
    tf.keras.optimizers.Adagrad,
    tf.keras.optimizers.AdamW,
    tf.keras.optimizers.Adamax,
    tf.keras.optimizers.Ftrl,
    tf.keras.optimizers.Lion,
    tf.keras.optimizers.Nadam,
    tf.keras.optimizers.RMSprop,
    tf.keras.optimizers.SGD
]

for opt in optimizers_list:
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=opt(learning_rate=0.001),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    #model.summary()

    history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=0)
    print(opt.__name__)
    plot_history(history)
    collect[opt.__name__] = history.history['val_sparse_categorical_accuracy']

In [ ]:
pd.DataFrame(collect).T.style.background_gradient(cmap='viridis')

## Activation Functions

In [ ]:
collect = {}

activation_list = [
    'linear',
    'relu',
    'tanh',
    'elu',
    'softplus',
    'sigmoid'
]

for act in activation_list:
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation=act),
        tf.keras.layers.Dense(64, activation=act),
        tf.keras.layers.Dense(10)
    ])

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    #model.summary()

    history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=0)
    print(act)
    plot_history(history)
    collect[act] = history.history['val_sparse_categorical_accuracy']

In [ ]:
pd.DataFrame(collect).T.style.background_gradient(cmap='viridis')

## Functional API

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model

input = Input(shape=(28, 28))

flatten = Flatten(input_shape=(28, 28))(input)

dense1 = Dense(units=128, activation='relu')(flatten)
dense2 = Dense(units=64, activation='tanh')(flatten)

conc = concatenate([dense1, dense2])

output = Dense(10)(conc)

model = Model(inputs=input, outputs=output)

model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=opt(learning_rate=0.001),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

model.summary()

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=0)
plot_history(history)